Imports

In [251]:
import pandas as pd
import numpy as np
import os
import pylab as pl
import scipy.optimize as opt

%matplotlib inline 
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [252]:
# offense data
off2017 = pd.read_csv('/content/2017 NFL Offense.csv')
off2017['Year'] = '2017'

off2018 = pd.read_csv('/content/2018 NFL Offense.csv')
off2018['Year'] = '2018'

off2019 = pd.read_csv('/content/2019 NFL Offense.csv')
off2019['Year'] = '2019'

off2020 = pd.read_csv('/content/2020 NFL Offense.csv')
off2020['Year'] = '2020'

off2021 = pd.read_csv('/content/2021 NFL Offense.csv')
off2021['Year'] = '2021'

off2022 = pd.read_csv('/content/2022 NFL Offense.csv')
off2022['Year'] = '2022'

# defense data

def2017 = pd.read_csv('/content/2017 NFL Defense.csv')
def2017['Year'] = '2017'

def2018 = pd.read_csv('/content/2018 NFL Defense.csv')
def2018['Year'] = '2018'

def2019 = pd.read_csv('/content/2019 NFL Defense.csv')
def2019['Year'] = '2019'

def2020 = pd.read_csv('/content/2020 NFL Defense.csv')
def2020['Year'] = '2020'

def2021 = pd.read_csv('/content/2021 NFL Defense.csv')
def2021['Year'] = '2021'

def2022 = pd.read_csv('/content/2022 NFL Defense.csv')
def2022['Year'] = '2022'


In [253]:
# Merge offense data together

offense_df = [off2017[1:33], off2018[1:33], off2019[1:33], off2020[1:33], off2021[1:33]]
offense_total = pd.concat(offense_df)

In [254]:
offense_total.columns = ['o_rank', 'team', 'o_games_played', 'o_points_for', 'o_total_yards',
       'o_plays', 'o_y/play', 'o_turnovers', 'o_fumbles_lost',
       'o_1st_d', 'o_completions', 'o_pass_attempts', 'o_pass_yards', 'o_pass_td',
       'o_int', 'o_pass_yd/att', 'o_passing_1st_d', 'o_rush_att', 'o_rush_yds',
       'o_rush_td', 'o_rush_yd/att', 'o_rush_1st_d', 'o_pe', 'o_pe_yds',
       'o_pe_1st_d', 'o_scoring_drives', 'o_to_%', 'o_expected_points', 'year']

In [255]:
offense_total.head()

,o_rank,team,o_games_played,o_points_for,o_total_yards,o_plays,o_y/play,o_turnovers,o_fumbles_lost,o_1st_d,...,o_rush_td,o_rush_yd/att,o_rush_1st_d,o_pe,o_pe_yds,o_pe_1st_d,o_scoring_drives,o_to_%,o_expected_points,year
1,1,Los Angeles Rams,16,478,5784,1000,5.8,21,14,311,...,17,4.3,97,107,922,29,45.3,9.5,-43.18,2017
2,2,New England Patriots,16,458,6307,1070,5.9,12,4,389,...,16,4.2,107,95,835,50,49.4,6.9,114.84,2017
3,3,Philadelphia Eagles,16,457,5852,1073,5.5,20,11,338,...,9,4.5,108,116,962,37,40.3,9.4,21.06,2017
4,4,New Orleans Saints,16,448,6259,1000,6.3,18,10,332,...,23,4.7,111,105,967,20,44.6,9.1,7.38,2017
5,5,Jacksonville Jaguars,16,417,5855,1078,5.4,23,10,332,...,18,4.3,101,112,936,38,34.7,11.1,-36.01,2017


In [256]:
 defense_df = [def2017[1:33], def2018[1:33], def2019[1:33], def2020[1:33], def2021[1:33]]
defense_total = pd.concat(defense_df)

In [257]:
defense_total.columns = ['d_rank', 'team', 'd_games_played', 'd_points_allowed', 'd_total_yds',
       'd_plays', 'd_y/play', 'd_td', 'd_fumbles_lost',
       'd_1st_d', 'd_pass_completions', 'd_pass_attempts', 'd_pass_yards', 'd_pass_td',
       'd_int', 'd_pass_yd/att', 'd_passing_1st_d', 'd_rush_att', 'def_rush_yds',
       'd_rush_td', 'd_rush_yds/att', 'd_rush_1st_d', 'd_pe', 'd_pe_yds',
       'd_pe_1st_d', 'd_scoring_drives', 'd_td_%', 'd_expected_points', 'year']

In [258]:
defense_total.head()

,d_rank,team,d_games_played,d_points_allowed,d_total_yds,d_plays,d_y/play,d_td,d_fumbles_lost,d_1st_d,...,d_rush_td,d_rush_yds/att,d_rush_1st_d,d_pe,d_pe_yds,d_pe_1st_d,d_scoring_drives,d_td_%,d_expected_points,year
1,1,Minnesota Vikings,16,252,4415,956,4.6,19,5,260,...,10,3.7,59,105,928,33,27.9,10.1,145.07,2017
2,2,Jacksonville Jaguars,16,268,4578,993,4.6,33,12,257,...,9,4.3,91,112,1050,23,23.9,15.6,252.15,2017
3,3,Los Angeles Chargers,16,272,5254,997,5.3,27,9,282,...,11,4.9,99,109,996,30,28.5,15.1,120.25,2017
4,4,Philadelphia Eagles,16,295,4904,976,5,31,12,272,...,7,3.8,62,101,917,33,27.1,15.4,162.09,2017
5,5,New England Patriots,16,296,5856,1022,5.7,18,6,325,...,6,4.7,98,111,1035,23,30.2,9.9,12.09,2017


In [259]:
train_set = offense_total.merge(defense_total, on = ['team', 'year'])

Test Set

In [260]:
off2022 = off2022[1:33]
off2022.columns = ['o_rank', 'team', 'o_games_played', 'o_points_for', 'o_total_yards',
       'o_plays', 'o_y/play', 'o_turnovers', 'o_fumbles_lost',
       'o_1st_d', 'o_completions', 'o_pass_attempts', 'o_pass_yards', 'o_pass_td',
       'o_int', 'o_pass_yd/att', 'o_passing_1st_d', 'o_rush_att', 'o_rush_yds',
       'o_rush_td', 'o_rush_yd/att', 'o_rush_1st_d', 'o_pe', 'o_pe_yds',
       'o_pe_1st_d', 'o_scoring_drives', 'o_to_%', 'o_expected_points', 'year']

def2022 = def2022[1:33]
def2022.columns = ['d_rank', 'team', 'd_games_played', 'd_points_allowed', 'd_total_yds',
       'd_plays', 'd_y/play', 'd_td', 'd_fumbles_lost',
       'd_1st_d', 'd_pass_completions', 'd_pass_attempts', 'd_pass_yards', 'd_pass_td',
       'd_int', 'd_pass_yd/att', 'd_passing_1st_d', 'd_rush_att', 'def_rush_yds',
       'd_rush_td', 'd_rush_yds/att', 'd_rush_1st_d', 'd_pe', 'd_pe_yds',
       'd_pe_1st_d', 'd_scoring_drives', 'd_td_%', 'd_expected_points', 'year']

test_set = off2022.merge(def2022, on = ['team', 'year'])
test_set.head()

,o_rank,team,o_games_played,o_points_for,o_total_yards,o_plays,o_y/play,o_turnovers,o_fumbles_lost,o_1st_d,...,def_rush_yds,d_rush_td,d_rush_yds/att,d_rush_1st_d,d_pe,d_pe_yds,d_pe_1st_d,d_scoring_drives,d_td_%,d_expected_points
0,1,Kansas City Chiefs,17,496,7032,1094,6.4,23,11,408,...,1823,10,4.4,102,98,734,28,35.9,10.3,-65.59
1,2,Philadelphia Eagles,17,477,6614,1124,5.9,19,10,385,...,2068,15,4.6,115,95,798,24,32.4,15.1,20.73
2,3,Dallas Cowboys,17,467,6034,1114,5.4,23,5,347,...,2198,9,4.4,119,98,851,34,33.5,16.2,31.77
3,4,Buffalo Bills,16,455,6361,1037,6.1,27,13,367,...,1673,10,4.3,103,89,754,29,31,14.3,4.46
4,5,Detroit Lions,17,453,6460,1092,5.9,15,8,376,...,2491,22,5.2,135,103,818,33,41.2,11.9,-159.66


In [261]:
# Converting columns to floats. Adjusting for games played to account for extended season, and in 2022 Bills and Cincinatti played 16 games instead of 17.

adj_columns = ['o_games_played', 'o_points_for', 'o_total_yards',
       'o_plays', 'o_y/play', 'o_turnovers', 'o_fumbles_lost',
       'o_1st_d', 'o_completions', 'o_pass_attempts', 'o_pass_yards', 'o_pass_td',
       'o_int', 'o_pass_yd/att', 'o_passing_1st_d', 'o_rush_att', 'o_rush_yds',
       'o_rush_td', 'o_rush_yd/att', 'o_rush_1st_d', 'o_pe', 'o_pe_yds',
       'o_pe_1st_d', 'o_scoring_drives', 'o_to_%', 'o_expected_points',
       'd_games_played', 'd_points_allowed', 'd_total_yds',
       'd_plays', 'd_y/play', 'd_td', 'd_fumbles_lost',
       'd_1st_d', 'd_pass_completions', 'd_pass_attempts', 'd_pass_yards', 'd_pass_td',
       'd_int', 'd_pass_yd/att', 'd_passing_1st_d', 'd_rush_att', 'def_rush_yds',
       'd_rush_td', 'd_rush_yds/att', 'd_rush_1st_d', 'd_pe', 'd_pe_yds',
       'd_pe_1st_d', 'd_scoring_drives', 'd_td_%', 'd_expected_points']

for col in [adj_columns]:
    test_set[col] = test_set[col].astype('float')

test_set['o_games_played'] = test_set['o_games_played'].astype('int')
adjuster = 17 / test_set['o_games_played'] 

test_set[adj_columns] = test_set[adj_columns].mul(adjuster, axis = 0)
test_set.head()

,o_rank,team,o_games_played,o_points_for,o_total_yards,o_plays,o_y/play,o_turnovers,o_fumbles_lost,o_1st_d,...,def_rush_yds,d_rush_td,d_rush_yds/att,d_rush_1st_d,d_pe,d_pe_yds,d_pe_1st_d,d_scoring_drives,d_td_%,d_expected_points
0,1,Kansas City Chiefs,17.0,496.0000,7032.0000,1094.0000,6.40000,23.0000,11.0000,408.0000,...,1823.0000,10.000,4.40000,102.0000,98.0000,734.000,28.0000,35.9000,10.30000,-65.59000
1,2,Philadelphia Eagles,17.0,477.0000,6614.0000,1124.0000,5.90000,19.0000,10.0000,385.0000,...,2068.0000,15.000,4.60000,115.0000,95.0000,798.000,24.0000,32.4000,15.10000,20.73000
2,3,Dallas Cowboys,17.0,467.0000,6034.0000,1114.0000,5.40000,23.0000,5.0000,347.0000,...,2198.0000,9.000,4.40000,119.0000,98.0000,851.000,34.0000,33.5000,16.20000,31.77000
3,4,Buffalo Bills,17.0,483.4375,6758.5625,1101.8125,6.48125,28.6875,13.8125,389.9375,...,1777.5625,10.625,4.56875,109.4375,94.5625,801.125,30.8125,32.9375,15.19375,4.73875
4,5,Detroit Lions,17.0,453.0000,6460.0000,1092.0000,5.90000,15.0000,8.0000,376.0000,...,2491.0000,22.000,5.20000,135.0000,103.0000,818.000,33.0000,41.2000,11.90000,-159.66000


In [262]:
all_data = pd.concat([test_set, train_set])

In [263]:
columns = ['o_games_played', 'o_points_for', 'o_total_yards',
       'o_plays', 'o_y/play', 'o_turnovers', 'o_fumbles_lost',
       'o_1st_d', 'o_completions', 'o_pass_attempts', 'o_pass_yards', 'o_pass_td',
       'o_int', 'o_pass_yd/att', 'o_passing_1st_d', 'o_rush_att', 'o_rush_yds',
       'o_rush_td', 'o_rush_yd/att', 'o_rush_1st_d', 'o_pe', 'o_pe_yds',
       'o_pe_1st_d', 'o_scoring_drives', 'o_to_%', 'o_expected_points',
       'd_games_played', 'd_points_allowed', 'd_total_yds',
       'd_plays', 'd_y/play', 'd_td', 'd_fumbles_lost',
       'd_1st_d', 'd_pass_completions', 'd_pass_attempts', 'd_pass_yards', 'd_pass_td',
       'd_int', 'd_pass_yd/att', 'd_passing_1st_d', 'd_rush_att', 'def_rush_yds',
       'd_rush_td', 'd_rush_yds/att', 'd_rush_1st_d', 'd_pe', 'd_pe_yds',
       'd_pe_1st_d', 'd_scoring_drives', 'd_td_%', 'd_expected_points']

for col in [columns]:
    all_data[col] = all_data[col].astype('float')

Adding Superbowl Winner Column

In [264]:
all_data['superbowl'] = 0

In [265]:
superbowl_winners = pd.read_csv('/content/SuperBowlWinners.csv')

In [266]:
superbowl_winners.Date = pd.to_datetime(superbowl_winners.Date)

In [267]:
superbowl_winners['year'] = pd.DatetimeIndex(superbowl_winners['Date']).year
superbowl_winners = superbowl_winners[['year', 'Winner']]
superbowl_winners.head()

,year,Winner
0,2023,Kansas City Chiefs
1,2022,Los Angeles Rams
2,2021,Tampa Bay Buccaneers
3,2020,Kansas City Chiefs
4,2019,New England Patriots


In [268]:
superbowl_winners['year'] = superbowl_winners['year'] - 1

In [269]:
superbowl_winners.year = superbowl_winners.year.astype('object')

In [270]:
superbowl_winners['superbowl'] = 1

In [271]:
superbowl_winners.head()

,year,Winner,superbowl
0,2022,Kansas City Chiefs,1
1,2021,Los Angeles Rams,1
2,2020,Tampa Bay Buccaneers,1
3,2019,Kansas City Chiefs,1
4,2018,New England Patriots,1


In [272]:
superbowl_winners.year = superbowl_winners.year.astype('int')
all_data.year = all_data.year.astype('int')

test = pd.merge(all_data, superbowl_winners, left_on = ['year','team'], right_on = ['year', 'Winner'], how = 'left')
test.head(32)

,o_rank,team,o_games_played,o_points_for,o_total_yards,o_plays,o_y/play,o_turnovers,o_fumbles_lost,o_1st_d,...,d_rush_1st_d,d_pe,d_pe_yds,d_pe_1st_d,d_scoring_drives,d_td_%,d_expected_points,superbowl_x,Winner,superbowl_y
0,1,Kansas City Chiefs,17.0,496.0000,7032.0000,1094.0000,6.40000,23.0000,11.0000,408.0000,...,102.0000,98.0000,734.000,28.0000,35.9000,10.30000,-65.59000,0,Kansas City Chiefs,1.0
1,2,Philadelphia Eagles,17.0,477.0000,6614.0000,1124.0000,5.90000,19.0000,10.0000,385.0000,...,115.0000,95.0000,798.000,24.0000,32.4000,15.10000,20.73000,0,NaN,NaN
2,3,Dallas Cowboys,17.0,467.0000,6034.0000,1114.0000,5.40000,23.0000,5.0000,347.0000,...,119.0000,98.0000,851.000,34.0000,33.5000,16.20000,31.77000,0,NaN,NaN
3,4,Buffalo Bills,17.0,483.4375,6758.5625,1101.8125,6.48125,28.6875,13.8125,389.9375,...,109.4375,94.5625,801.125,30.8125,32.9375,15.19375,4.73875,0,NaN,NaN
4,5,Detroit Lions,17.0,453.0000,6460.0000,1092.0000,5.90000,15.0000,8.0000,376.0000,...,135.0000,103.0000,818.000,33.0000,41.2000,11.90000,-159.66000,0,NaN,NaN
5,6,San Francisco 49ers,17.0,450.0000,6216.0000,1047.0000,5.90000,17.0000,8.0000,347.0000,...,77.0000,96.0000,832.000,26.0000,25.7000,15.30000,56.26000,0,NaN,NaN
6,7,Minnesota Vikings,17.0,424.0000,6145.0000,1123.0000,5.50000,23.0000,8.0000,382.0000,...,113.0000,111.0000,926.000,22.0000,41.1000,12.00000,-104.00000,0,NaN,NaN
7,8,Cincinnati Bengals,17.0,444.1250,6128.5000,1118.8125,5.84375,19.1250,6.3750,379.3125,...,108.3750,106.2500,884.000,21.2500,38.6750,13.91875,-21.27125,0,NaN,NaN
8,9,Seattle Seahawks,17.0,407.0000,5976.0000,1044.0000,5.70000,23.0000,11.0000,346.0000,...,130.0000,98.0000,818.000,29.0000,37.2000,12.20000,-113.17000,0,NaN,NaN
9,10,Jacksonville Jaguars,17.0,404.0000,6075.0000,1072.0000,5.70000,22.0000,13.0000,352.0000,...,114.0000,90.0000,683.000,25.0000,36.1000,14.40000,-52.27000,0,NaN,NaN


In [273]:
test.drop(['superbowl_x', 'Winner'], axis=1, inplace = True)

In [274]:
test.superbowl_y = test.superbowl_y.fillna(0)
test.superbowl_y = test.superbowl_y.astype('int')
test.rename(columns = {'superbowl_y': 'superbowl'}, inplace = True)

In [275]:
test.head()

,o_rank,team,o_games_played,o_points_for,o_total_yards,o_plays,o_y/play,o_turnovers,o_fumbles_lost,o_1st_d,...,d_rush_td,d_rush_yds/att,d_rush_1st_d,d_pe,d_pe_yds,d_pe_1st_d,d_scoring_drives,d_td_%,d_expected_points,superbowl
0,1,Kansas City Chiefs,17.0,496.0000,7032.0000,1094.0000,6.40000,23.0000,11.0000,408.0000,...,10.000,4.40000,102.0000,98.0000,734.000,28.0000,35.9000,10.30000,-65.59000,1
1,2,Philadelphia Eagles,17.0,477.0000,6614.0000,1124.0000,5.90000,19.0000,10.0000,385.0000,...,15.000,4.60000,115.0000,95.0000,798.000,24.0000,32.4000,15.10000,20.73000,0
2,3,Dallas Cowboys,17.0,467.0000,6034.0000,1114.0000,5.40000,23.0000,5.0000,347.0000,...,9.000,4.40000,119.0000,98.0000,851.000,34.0000,33.5000,16.20000,31.77000,0
3,4,Buffalo Bills,17.0,483.4375,6758.5625,1101.8125,6.48125,28.6875,13.8125,389.9375,...,10.625,4.56875,109.4375,94.5625,801.125,30.8125,32.9375,15.19375,4.73875,0
4,5,Detroit Lions,17.0,453.0000,6460.0000,1092.0000,5.90000,15.0000,8.0000,376.0000,...,22.000,5.20000,135.0000,103.0000,818.000,33.0000,41.2000,11.90000,-159.66000,0


In [276]:
test.dtypes
test.superbowl.sum()

6

In [277]:
test.columns

Index(['o_rank', 'team', 'o_games_played', 'o_points_for', 'o_total_yards',
       'o_plays', 'o_y/play', 'o_turnovers', 'o_fumbles_lost', 'o_1st_d',
       'o_completions', 'o_pass_attempts', 'o_pass_yards', 'o_pass_td',
       'o_int', 'o_pass_yd/att', 'o_passing_1st_d', 'o_rush_att', 'o_rush_yds',
       'o_rush_td', 'o_rush_yd/att', 'o_rush_1st_d', 'o_pe', 'o_pe_yds',
       'o_pe_1st_d', 'o_scoring_drives', 'o_to_%', 'o_expected_points', 'year',
       'd_rank', 'd_games_played', 'd_points_allowed', 'd_total_yds',
       'd_plays', 'd_y/play', 'd_td', 'd_fumbles_lost', 'd_1st_d',
       'd_pass_completions', 'd_pass_attempts', 'd_pass_yards', 'd_pass_td',
       'd_int', 'd_pass_yd/att', 'd_passing_1st_d', 'd_rush_att',
       'def_rush_yds', 'd_rush_td', 'd_rush_yds/att', 'd_rush_1st_d', 'd_pe',
       'd_pe_yds', 'd_pe_1st_d', 'd_scoring_drives', 'd_td_%',
       'd_expected_points', 'superbowl'],
      dtype='object')

Modeling

In [278]:
#split into values for test and train

test[['o_rank', 'd_rank']] = test[['o_rank', 'd_rank']].astype('int')

train_columns = ['o_rank', 'o_points_for', 'o_total_yards',
       'o_y/play', 'o_turnovers', 'o_1st_d',
       'o_completions', 'o_pass_yards', 'o_pass_td', 'o_int',
       'o_pass_yd/att', 'o_passing_1st_d', 'o_rush_yds',
       'o_rush_td', 'o_rush_yd/att', 'o_rush_1st_d', 'o_pe', 'o_pe_yds', 
       'o_scoring_drives', 'd_rank',
       'd_points_allowed', 'd_total_yds', 
       'd_y/play', 'd_td', 'd_fumbles_lost', 'd_1st_d', 'd_pass_completions',
       'd_pass_yards', 'd_pass_td',
       'd_int', 'd_pass_yd/att', 'd_passing_1st_d', 'def_rush_yds', 'd_rush_td', 'd_rush_yds/att', 'd_rush_1st_d', 'd_pe',
       'd_pe_yds', 'd_scoring_drives']

X = np.asarray(test[train_columns])
y = np.asarray(test['superbowl'])
print(X.shape)
print(y.shape)

(192, 39)
(192,)


In [279]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fitting the StandardScaler to the training data
scaler = preprocessing.StandardScaler().fit(X_train)

# Scaling the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Getting the team, offensive rank, and defensive rank data for the first 32 rows
teams = test.loc[:31, 'team']
o_rank = test.loc[:31, 'o_rank']
d_rank = test.loc[:31, 'd_rank']

print(X_test.shape)
print(y_test.shape)
print(X_train.shape)
print(y_train.shape)

(39, 39)
(39,)
(153, 39)
(153,)


Logistic Regression

In [281]:
# Training the model

LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train, y_train)

# Predict classes and probabilities
yhat = LR.predict(X_test)
yhat_prob = LR.predict_proba(X_test)[:, 1]

# Compute accuracy on training set
train_accuracy = accuracy_score(y_train, LR.predict(X_train))
print("Accuracy:", train_accuracy)

Accuracy: 0.9869281045751634


In [284]:
# Create and sort the prediction DataFrame
df2 = pd.DataFrame({'Team':teams[:32], 'Prediction':yhat_prob[:32], 'o_rank':o_rank[:32], 'd_rank':d_rank[:32]}) 
df2 = df2.sort_values(by='Prediction', ascending=False)
df2['Prediction_Rank'] = df2['Prediction'].rank(ascending=False)

df2

,Team,Prediction,o_rank,d_rank,Prediction_Rank
31,Denver Broncos,8.352211e-01,32,14,1.0
23,Washington Commanders,4.888543e-01,24,7,2.0
13,Green Bay Packers,6.170815e-02,14,17,3.0
12,Los Angeles Chargers,7.293144e-03,13,21,4.0
19,Carolina Panthers,4.859351e-03,20,19,5.0
24,Tampa Bay Buccaneers,2.750257e-03,25,13,6.0
1,Philadelphia Eagles,1.417198e-03,2,8,7.0
3,Buffalo Bills,8.711246e-04,4,2,8.0
17,Cleveland Browns,1.021457e-04,18,20,9.0
2,Dallas Cowboys,6.975820e-05,3,6,10.0
